In [ ]:
# all imports for this notebook
import intake
import satsearch
from satstac import ItemCollection
from lxml import objectify, etree

1. requete PEPS (Sentinel-Hub) avec intake-STAC

In [ ]:
#Q? find catalog.json under these URLs
sentinel_hub_catalogue = "https://services.sentinel-hub.com/api/v1/catalog/"
sentinel_hub_search = "https://services.sentinel-hub.com/api/v1/catalog/search"

Landsat-8 example

In [ ]:
landsat8_url = "https://raw.githubusercontent.com/sat-utils/sat-stac/master/test/catalog/catalog.json"
cat = intake.open_stac_catalog(landsat8_url)
list(cat)

In [ ]:
cat.metadata

In [ ]:
list(cat['stac-catalog-eo'])

In [ ]:
ls8 = cat['stac-catalog-eo']['landsat-8-l1']
list(ls8)

In [ ]:
ls8_item = ls8['LC08_L1TP_152038_20200611_20200611_01_RT']
list(ls8_item)

In [ ]:
ls8_item.metadata

In [ ]:
cat.search('instruments')

SAT-SEARCH 
Trying: Search functionality over STAC APIs: https://github.com/sat-utils/sat-search/blob/master/tutorial-1.ipynb

In [ ]:
URL =  "https://earth-search.aws.element84.com/v0"
results = satsearch.Search.search(url=URL,
                                bbox=[-110, 39.5, -105, 40.5],
                               datetime='2018-02-12T00:00:00Z/2018-03-18T12:31:12Z',
                               query=["eo:cloud_cover<10"],
                               collections=['sentinel-s2-l2a'])

print('Found %s items' % results.found())                                  
items = results.items(limit=5)
print('Summary: %s' % items.summary())

print('%s collections' % len(items._collections))
col = items._collections[0]
print('Collection: %s' % col)
#for key in col.keys():
#    print('%s: %s' % (key, col[key]))
    
print('\nITEMS: ')
for item in items:
    print('\nid: ',item.id)
    print('coll: ', item.collection())
    
    print("\nPROPERTIES:")
    print(item.properties.keys())
    for key in item.properties:
        print('%s: %s' % (key, item.properties[key]))
    
    print("\nASSETS: ")
    print(item.assets.keys())
    for key in item.assets.keys():
        print('%s: %s' % (key, item.assets[key]))
    break #just print first


In [ ]:
#save and load items collection to json
items.save('test.json')
items2 = ItemCollection.open('test.json')

print(items2.summary(['date', 'id', 'eo:cloud_cover']))

In [ ]:
# download a specific asset from all items and put in a directory by date in 'downloads'
filenames = items.download('metadata', filename_template='downloads/${id}')
print(filenames)

In [ ]:
# inspect a metadata file
input_file = 'downloads/S2B_12SXJ_20180316_1_L2A_metadata.xml'
root = objectify.parse(input_file)
print(etree.tostring(root))

INTAKE-ESM

In [ ]:
col_url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)
col

In [ ]:
col_subset = col.search(
         experiment_id=["historical", "ssp585"],
         table_id="Oyr",
         variable_id="o2",
         grid_label="gn",
)
col_subset